In [243]:
#libraries
import numpy as np 
import pandas as pd

**Вариант 2. Задание 10.**

\begin{equation*}
 \begin{cases}
    \frac{\partial u}{\partial t} = \frac{\partial}{r \partial r}(ru^{1/2}\frac{\partial u}{\partial r}) + \frac{\partial}{r^2 \partial \varphi}(u^{1/2}\frac{\partial u}{\partial \varphi}) , 0 < t,r \le 1, 0 < \varphi < \pi / 2 \\
    u(0, r, \varphi) = \frac{r^4cos^4(\varphi)}{121}, 0 \le r \le 1, 0 \le \varphi \le \pi / 2 \\
    u(t. 0, \varphi) = 0, 0 < t \le 1, 0 \le \varphi \le \pi / 2\\
    u(t, 1, \varphi) = \frac{cos^4(\varphi)}{(11 - 10t)^2}, 0 < t \le 1, 0 \le \varphi < \pi / 2 \\
    u(t, r, 0) = \frac{r^4}{(11 - 10t)^2}, 0 < r < 1, 0 < t \le 1 \\
    u(t, r, \pi / 2) = 0, 0 < r < 1, 0 < t \le 1    
 \end{cases}
\end{equation*}

In [244]:
def f_private(t, r, phi):
    '''
    функция частного решения
    '''
    return r**4 * (np.cos(phi))**4 / (11 - 10 * t)**2

def private_solution(t, r, phi):
    '''
    частное решение на сетке
    '''
    u = np.zeros((len(phi), len(r)))
    for i in range(len(phi)):
        u[i, :] = f_private(t, r, phi[i])
    return u

In [245]:
def grid(L, M, N):
    '''
    returns np.arrays of x, t broken into L and N pieces
    '''
    return np.linspace(0, 1, L), np.linspace(0, np.pi /2, M), np.linspace(0, 1, N)#менять первый и второй параметр, если изменятся границы

def sweep_mthd(a, b, c, d, m, l, u_ex):
    '''
    Метод прогонки
    '''
    alpha = np.zeros((m, l))
    beta = np.zeros((m, l))
    # beta[1:-1, 0] = u_ex[1:-1,0]
    x = u_ex.copy()
    
    for i in range(1, l - 1):
        alpha[1:-1,i] = (-a(i) / (c(i) * alpha[1:-1,i-1] + b(i)))
        beta[1:-1,i] = (d(i) - c(i) * beta[1:-1,i-1]) / (b(i) + c(i) * alpha[1:-1,i-1])
    
    for i in reversed(range(1, l-1)):
        x[1:-1,i] = alpha[1:-1,i] * x[1:-1,i + 1]  + beta[1:-1,i]

    return x

def accuracy(u1, u2):
    '''
    Вычислят точность
    Returns bool
    '''
    max = 0
    epsilon = 0.01#точность
    for m in range(u1.shape[1]):
        for l in range(u1.shape[0]):
            if u2[m][l] != 0:
                num = abs((u2[m][l] - u1[m][l]) / u2[m][l])
                if num > max:
                    max = num
            if max > epsilon:
                return False
    return True

def first_step(u, u1, r, m, n, mu, tau, hr):
    
    def a_l(l):
        return -hr * (l + 0.5) * (u1[1:-1, l + 1]**mu + u1[1:-1, l]**mu) * tau / (2 * r[l] * hr**2)
    def c_l(l):
        return -hr * (l - 0.5) * (u1[1:-1, l]**mu + u1[1:-1, l - 1]**mu) * tau / (2 * r[l] * hr**2)
    def b_l(l):    
        return 1 - a_l(l) - c_l(l)
    def d_l(l):
        return u[n, 1:- 1, l]
    
    return sweep_mthd(a_l, b_l, c_l, d_l, m, len(r), u[n+1,:,:])

def second_step(u, u_tilda, u_f, r, m, mu, tau, hf):

    u1 = u
    u_tilda1 = u_tilda.T

    def a_m(m):
        return -(u1[1:-1, m + 1]**mu + u1[1:-1, m]**mu) * tau / (2 * (r[1:-1] * hf)**2)
    def c_m(m):
        return -(u1[1:-1, m]**mu + u1[1:-1, m - 1]**mu) * tau / (2 * (r[1:-1] * hf)**2)
    def b_m(m):
        return 1 - a_m(m) - c_m(m)
    def d_m(m):
        return u_tilda1[1:-1, m]
    
    return sweep_mthd(a_m, b_m, c_m, d_m, len(r), m, u_f.T).T

    
    
def numerical_solve(r, phi, t, mu):
    hr = 1 / (len(r) - 1)
    hf = np.pi / 2 / (len(phi) - 1)
    tau = 1 / (len(t) - 1)
    
    m = len(phi)
    
    u = np.zeros((len(t), len(phi), len(r)))
    
    for i in range(len(phi)):#первое граничное условие
        u[0, i, :] = f_private(0, r, phi[i])
    
    for i in range(1, len(t)):
        u[i, :, -1] = f_private(t[i], r[-1], phi)#третье граничное условие
        u[i, 0, 1:-1] = f_private(t[i], r[1:-1], phi[0])#четвертое граничное условие
   
    for i in range(len(t) - 1):
        u_smth = u[i].copy()
        
        while True:
            u_tilda = first_step(u, u_smth, r, m, i, mu, tau, hr)
            u_end = second_step(u_smth, u_tilda, u[i+1], r, m, mu, tau, hf)
            if accuracy(u_smth, u_end):
                u[i + 1] = u_end
                break
            else:
                u_smth = u_end
             
    return u

In [246]:
#ТО ШО МЕНЯТЬ
L = M = 81
N = 1281
mu = 0.5
T = 1

#находим аналитическое решение
r_rep, phi_rep, _ = grid(6, 6, 6) 

u_private = private_solution(T, r_rep, phi_rep)


#находим численное рашение
r, phi, t = grid(L, M, N)
u_n = numerical_solve(r, phi, t, mu)
u_numeric = u_n[-1,::16, ::16]


    
print('Max error:', np.max(np.abs(u_private - u_numeric)))



Max error: 0.022559170768037956


In [247]:
print('                       Analytical                    ')
f = pd.DataFrame(u_private, columns=r_rep, index=phi_rep)
f

                       Analytical                    


,0.0,0.2,0.4,0.6,0.8,1.0
0.000000,0.0,1.600000e-03,2.560000e-02,1.296000e-01,4.096000e-01,1.000000e+00
0.314159,0.0,1.309017e-03,2.094427e-02,1.060304e-01,3.351084e-01,8.181356e-01
0.628319,0.0,6.854102e-04,1.096656e-02,5.551823e-02,1.754650e-01,4.283814e-01
0.942478,0.0,1.909830e-04,3.055728e-03,1.546962e-02,4.889165e-02,1.193644e-01
1.256637,0.0,1.458980e-05,2.334369e-04,1.181774e-03,3.734990e-03,9.118627e-03
1.570796,0.0,2.249279e-68,3.598847e-67,1.821916e-66,5.758155e-66,1.405800e-65


In [248]:
print('                         Numerical                   ')
f1 = pd.DataFrame(u_numeric, columns=r_rep, index=phi_rep)
f1


                         Numerical                   


,0.0,0.2,0.4,0.6,0.8,1.0
0.000000,0.0,0.001600,0.025600,0.129600,0.409600,1.000000e+00
0.314159,0.0,0.001503,0.025092,0.118485,0.350898,8.181356e-01
0.628319,0.0,0.000440,0.005505,0.038211,0.152906,4.283814e-01
0.942478,0.0,0.000171,0.001009,0.003914,0.026600,1.193644e-01
1.256637,0.0,0.000061,0.000242,0.000285,0.000489,9.118627e-03
1.570796,0.0,0.000000,0.000000,0.000000,0.000000,1.405800e-65


In [249]:
print('                          Errors                          ')
fe =  pd.DataFrame(np.abs(u_private - u_numeric), columns=r_rep, index=phi_rep)
fe

                          Errors                          


,0.0,0.2,0.4,0.6,0.8,1.0
0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
0.314159,0.0,1.941282e-04,4.147838e-03,1.245415e-02,1.579002e-02,0.0
0.628319,0.0,2.453662e-04,5.461683e-03,1.730771e-02,2.255917e-02,0.0
0.942478,0.0,2.033000e-05,2.047065e-03,1.155584e-02,2.229141e-02,0.0
1.256637,0.0,4.600396e-05,8.140463e-06,8.972166e-04,3.245837e-03,0.0
1.570796,0.0,2.249279e-68,3.598847e-67,1.821916e-66,5.758155e-66,0.0
